In [ ]:
import os
import cv2
import time
import datetime
import numpy as np
import pandas as pd
from imutils import paths
import pickle

try:
    import face_recognition
except Exception as e:
    raise ImportError("face_recognition library is required. Install with: pip install face_recognition")

from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

try:
    import tensorflow as tf
    from keras import layers, models
except Exception as e:
    tf = None

DATA_DIR = "dataset_faces"
EMOTION_MODEL_PATH = "models/emotion_model.keras"
FACE_ENCODER_PATH = "models/face_encoder.pkl"
FACE_CLASSIFIER_PATH = "models/face_classifier.pkl"
ATTENDANCE_CSV = "datasets/attendance.csv"

os.makedirs(DATA_DIR, exist_ok=True)

2025-09-05 23:28:03.259536: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-05 23:28:03.259803: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 23:28:03.344387: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-05 23:28:07.910301: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
def collect_images(name, num_images=60, delay=0.2):
    """Capture images of a student from webcam and save to dataset directory."""
    person_dir = os.path.join(DATA_DIR, name)
    os.makedirs(person_dir, exist_ok=True)

    cap = cv2.VideoCapture(0)
    print(f"Starting capture for '{name}'. Press 'q' to stop early.")
    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        cv2.putText(frame, f"Capture: {count}/{num_images}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.imshow("Collect Images - Press q to quit", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

        if count < num_images:
            filename = os.path.join(person_dir, f"{name}_{int(time.time()*1000)}_{count}.jpg")
            cv2.imwrite(filename, frame)
            count += 1
            time.sleep(delay)
        else:
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Captured {count} images for {name} at {person_dir}")

collect_images("alice")

Starting capture for 'alice'. Press 'q' to stop early.


Captured 60 images for alice at dataset_faces/alice


In [4]:
def train_face_classifier():
    """Compute face embeddings using face_recognition and train an SVM classifier."""
    print("Loading images and computing embeddings. This may take a while...")
    imagePaths = list(paths.list_images(DATA_DIR))
    knownEncodings = []
    knownNames = []

    for (i, imagePath) in enumerate(imagePaths):
        name = imagePath.split(os.path.sep)[-2]
        image = cv2.imread(imagePath)
        if image is None:
            continue
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        boxes = face_recognition.face_locations(rgb, model='hog')
        encodings = face_recognition.face_encodings(rgb, boxes)
        if len(encodings) == 0:
            continue
        knownEncodings.append(encodings[0])
        knownNames.append(name)

    if len(knownEncodings) == 0:
        print("No face encodings found. Make sure dataset is collected and faces are visible.")
        return

    le = LabelEncoder()
    y = le.fit_transform(knownNames)

    clf = SVC(C=1.0, kernel='linear', probability=True)
    clf.fit(knownEncodings, y)

    # save encoder and classifier
    with open(FACE_ENCODER_PATH, 'wb') as f:
        pickle.dump(le, f)
    with open(FACE_CLASSIFIER_PATH, 'wb') as f:
        pickle.dump(clf, f)

    print("Face classifier trained and saved:")
    print(" -", FACE_ENCODER_PATH)
    print(" -", FACE_CLASSIFIER_PATH)

train_face_classifier()

Loading images and computing embeddings. This may take a while...
Face classifier trained and saved:
 - models/face_encoder.pkl
 - models/face_classifier.pkl


In [6]:
def build_emotion_model(input_shape=(48,48,1), n_classes=7):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(32, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
def load_fer_csv(csv_path):
    df = pd.read_csv(csv_path)
    X = []
    y = []
    for idx, row in df.iterrows():
        pixels = row['pixels'].split()
        img = np.asarray(pixels, dtype='uint8').reshape(48,48)
        X.append(img)
        y.append(row['emotion'])
    X = np.array(X)
    y = np.array(y)
    X = X.astype('float32') / 255.0
    X = np.expand_dims(X, -1)
    return X, y

In [9]:
def train_emotion_model(csv_path, epochs=20, batch_size=64):
    if tf is None:
        print("TensorFlow not available. Install with: pip install tensorflow")
        return
    print("Loading FER dataset from:", csv_path)
    X, y = load_fer_csv(csv_path)
    split = int(0.8 * len(X))
    X_train, X_val = X[:split], X[split:]
    y_train, y_val = y[:split], y[split:]

    model = build_emotion_model(input_shape=X.shape[1:], n_classes=len(np.unique(y)))
    model.summary()
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)
    model.save(EMOTION_MODEL_PATH)
    print("Emotion model trained and saved to", EMOTION_MODEL_PATH)

In [12]:
def recognize_and_mark_attendance(time_window=(datetime.time(9,30), datetime.time(10,30))):
    if not os.path.exists(FACE_ENCODER_PATH) or not os.path.exists(FACE_CLASSIFIER_PATH):
        print("Face classifier not found. Run 'train_face' first.")
        return

    with open(FACE_ENCODER_PATH, 'rb') as f:
        le = pickle.load(f)
    with open(FACE_CLASSIFIER_PATH, 'rb') as f:
        clf = pickle.load(f)

    emotion_model = None
    if os.path.exists(EMOTION_MODEL_PATH) and tf is not None:
        try:
            emotion_model = models.load_model(EMOTION_MODEL_PATH)
            print("Loaded emotion model.")
        except Exception as e:
            print("Could not load emotion model:", e)
            emotion_model = None

    start_t, end_t = time_window
    now = datetime.datetime.now().time()
    if not (start_t <= now <= end_t):
        print(f"Current time {now}. Outside attendance window {start_t} - {end_t}.")
        return

    cap = cv2.VideoCapture(0)
    attendance = {} 
    print("Attendance running. Press 'q' to stop early.")

    emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes = face_recognition.face_locations(rgb, model='hog')
        encodings = face_recognition.face_encodings(rgb, boxes)

        for (box, encoding) in zip(boxes, encodings):
            probs = clf.predict_proba([encoding])[0]
            idx = np.argmax(probs)
            name = le.inverse_transform([idx])[0]
            confidence = probs[idx]

            top_emotion = 'Unknown'
            (top, right, bottom, left) = box
            face_img = frame[top:bottom, left:right]
            if face_img.size != 0 and emotion_model is not None:
                gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
                gray = cv2.resize(gray, (48,48))
                inp = gray.astype('float32')/255.0
                inp = np.expand_dims(inp, axis=(0,-1))
                pred = emotion_model.predict(inp)
                eidx = np.argmax(pred)
                top_emotion = emotion_labels[eidx] if eidx < len(emotion_labels) else 'Unknown'

            ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if name not in attendance:
                attendance[name] = {'first_seen': ts, 'last_emotion': top_emotion, 'confidence': float(confidence)}
            else:
                attendance[name]['last_emotion'] = top_emotion
                attendance[name]['confidence'] = float(confidence)

            cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
            cv2.putText(frame, f"{name} {confidence:.2f}", (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),2)
            cv2.putText(frame, f"{top_emotion}", (left, bottom+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0),2)

        cv2.imshow("Attendance", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    rows = []
    for name, info in attendance.items():
        rows.append({
            'Name': name,
            'Status': 'Present',
            'FirstSeen': info['first_seen'],
            'LastEmotion': info['last_emotion'],
            'Confidence': info.get('confidence', None)
        })

    df = pd.DataFrame(rows, columns=['Name','Status','FirstSeen','LastEmotion','Confidence'])
    all_students = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
    for s in all_students:
        if s not in df['Name'].values:
            df = df._append({'Name': s, 'Status': 'Absent', 'FirstSeen': None, 'LastEmotion': None, 'Confidence': None}, ignore_index=True)

    df.to_csv(ATTENDANCE_CSV, index=False)
    print(f"Attendance saved to {ATTENDANCE_CSV}")

recognize_and_mark_attendance()

E0000 00:00:1757095357.097937   22114 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1757095357.108356   22114 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loaded emotion model.
Current time 23:32:37.976382. Outside attendance window 09:30:00 - 10:30:00.
